In [1]:
import polars as pl
import pandas as pd
import numpy as np
import os
import itertools
import warnings

In [2]:
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
path_dt = r"C:\Users\e_koffie\Documents\IAI_Project\IAI_PROJECT\Data\matrices_distance_temps\dt_matrice_ligne.parquet"

In [4]:
df_polars = pl.read_parquet(path_dt)
df_polars = df_polars.rename({"gid": "temps_parcours"})

In [5]:
df_polars = df_polars.unique()

In [6]:
unique_ids = pl.concat([
    df_polars.select(pl.col("Idloc_start").cast(pl.Utf8)).to_series(),
    df_polars.select(pl.col("Idloc_end").cast(pl.Utf8)).to_series()
]).unique().sort()

In [7]:
assert unique_ids.len() == 8396, f"Expected 8396 unique elements, got {unique_ids.len()}"

In [8]:
len(unique_ids)

8396

In [9]:
combinations = list(itertools.combinations(unique_ids.to_list(), 2))

In [10]:
result_df = pl.DataFrame(
    {
        "Idloc_start": [start for start, end in combinations],
        "Idloc_end": [end for start, end in combinations]
    }
)

In [11]:
df = df_polars.with_columns([
    pl.col("Idloc_start").cast(pl.Utf8),
    pl.col("Idloc_end").cast(pl.Utf8)
])
result_df = result_df.with_columns([
    pl.col("Idloc_start").cast(pl.Utf8),
    pl.col("Idloc_end").cast(pl.Utf8)
])

In [12]:
df_key = df.select(["Idloc_start", "Idloc_end"])

In [13]:
only_in_result_df = result_df.join(df_key, on=["Idloc_start", "Idloc_end"], how="anti")
only_in_df = df_key.join(result_df, on=["Idloc_start", "Idloc_end"], how="anti")

In [14]:
matching_df = df.join(result_df, on=["Idloc_start", "Idloc_end"], how="inner")

In [15]:
print("Only in result_df:", only_in_result_df.shape)
print("Only in df:", only_in_df.shape)
print("Matching rows:", matching_df.shape)

Only in result_df: (107, 2)
Only in df: (0, 2)
Matching rows: (35242103, 3)


In [16]:
result_df.height

35242210

In [17]:
df_polars.height

35242103

In [18]:
matching_df = matching_df.unique()

In [19]:
matching_df

Idloc_start,Idloc_end,temps_parcours
str,str,f64
"""020270598008""","""030291601003""",1.3878e6
"""080700301004""","""150210698007""",644660.373208
"""150300498041""","""180580601012""",922306.465015
"""080890498005""","""210610501005""",351897.620503
"""010020498003""","""210610501007""",486248.187976
…,…,…
"""050060101004""","""100340498001""",1.2123e6
"""020480198006""","""310780198002""",748110.461237
"""030291398019""","""250380398001""",1.1991e6


In [20]:
only_in_result_df

Idloc_start,Idloc_end
str,str
"""010020298003""","""010020298004"""
"""020270398012""","""020270398013"""
"""020480298014""","""020480298016"""
"""020480298017""","""020480298996"""
"""020950298001""","""020950298002"""
…,…
"""321010201003""","""321010201997"""
"""321010298012""","""321010298014"""
"""330710198005""","""330710198006"""


In [21]:
# Read the route file using pandas
route_df = pd.read_excel(
    r"C:\Users\e_koffie\Documents\IAI_Project\IAI_PROJECT\Data\Fichiers_Routes_Localites\localite_noeuds_02102024_VF_2.xls",
dtype={"idloc_iai": str})

# Ensure idloc_iai and id_code are strings for matching
route_df["idloc_iai"] = route_df["idloc_iai"].astype(str)
route_df["id_code"] = route_df["id"].astype(str)

In [22]:
# Create mapping from idloc_iai to id_code
idloc_to_code = dict(zip(route_df["idloc_iai"], route_df["id_code"]))

In [23]:
not_matching = only_in_result_df.with_columns([
    pl.col("Idloc_start").cast(str).replace(idloc_to_code, default=None).cast(pl.Int64).alias("id_start"),
    pl.col("Idloc_end").cast(str).replace(idloc_to_code, default=None).cast(pl.Int64).alias("id_end")
])

C:\Users\e_koffie\AppData\Local\Temp\ipykernel_20912\63992755.py:2: DeprecationWarning: The `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
  pl.col("Idloc_start").cast(str).replace(idloc_to_code, default=None).cast(pl.Int64).alias("id_start"),
C:\Users\e_koffie\AppData\Local\Temp\ipykernel_20912\63992755.py:3: DeprecationWarning: The `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
  pl.col("Idloc_end").cast(str).replace(idloc_to_code, default=None).cast(pl.Int64).alias("id_end")


In [24]:
not_matching

Idloc_start,Idloc_end,id_start,id_end
str,str,i64,i64
"""010020298003""","""010020298004""",10421,15005
"""020270398012""","""020270398013""",10470,14732
"""020480298014""","""020480298016""",10163,10162
"""020480298017""","""020480298996""",33104,33260
"""020950298001""","""020950298002""",34016,29816
…,…,…,…
"""321010201003""","""321010201997""",11632,10915
"""321010298012""","""321010298014""",3899,25293
"""330710198005""","""330710198006""",13403,14863


In [25]:
# Permute Idloc_start with Idloc_end and id_start with id_end, then concatenate with original not_matching
not_matching_permuted = not_matching.select([
    pl.col("Idloc_end").alias("Idloc_start"),
    pl.col("Idloc_start").alias("Idloc_end"),
    pl.col("id_end").alias("id_start"),
    pl.col("id_start").alias("id_end"),
])

# Concatenate original and permuted
not_matching_concat = pl.concat([not_matching, not_matching_permuted])
not_matching_concat

Idloc_start,Idloc_end,id_start,id_end
str,str,i64,i64
"""010020298003""","""010020298004""",10421,15005
"""020270398012""","""020270398013""",10470,14732
"""020480298014""","""020480298016""",10163,10162
"""020480298017""","""020480298996""",33104,33260
"""020950298001""","""020950298002""",34016,29816
…,…,…,…
"""321010201997""","""321010201003""",10915,11632
"""321010298014""","""321010298012""",25293,3899
"""330710198006""","""330710198005""",14863,13403


In [26]:
from tqdm import tqdm
from pathlib import Path
import os

def check_combinations(matrix_root: str, not_matching_concat: pl.DataFrame) -> pl.DataFrame:
    """
    Vérifie pour chaque combinaison (id_start, id_end) dans not_matching_concat
    si elle existe dans l'une des matrices txt du dossier matrix_root.

    Retourne not_matching_concat avec deux colonnes supplémentaires :
    - 'exists_in_matrix': True si trouvé
    - 'matrix_path': chemin vers la première matrice contenant la paire
    """
    # Préparer les combinaisons à rechercher
    not_matching_concat = not_matching_concat.with_columns([
        pl.struct(["id_start", "id_end"]).alias("combination")
    ])

    exists = [False] * len(not_matching_concat)
    paths = [None] * len(not_matching_concat)

    # Obtenir les fichiers txt
    txt_files = list(Path(matrix_root).rglob("*.txt"))
    txt_files2 = list(Path(r"E:\matrice-ligne-diarra").rglob("*.txt"))

    txt_files = txt_files + txt_files2

    # Indexation rapide des combinaisons à vérifier
    combo_to_idx = {
        (row[0], row[1]): i
        for i, row in enumerate(not_matching_concat.select(["id_start", "id_end"]).rows())
        }   
    found_set = set()

    print(f"🔍 Analyse de {len(txt_files)} matrices pour {len(combo_to_idx)} combinaisons...")

    # Parcours des matrices avec progress bar
    for txt_path in tqdm(txt_files, desc="Lecture des matrices .txt", unit="fichier"):
        try:
            df = pl.read_csv(
                txt_path,
                separator=";",
                dtypes={"start_vid": pl.Int64, "end_vid": pl.Int64},
                ignore_errors=True
            )
        except Exception as e:
            print(f"Erreur lors de la lecture de {txt_path}: {e}")
            continue

        if "start_vid" not in df.columns or "end_vid" not in df.columns:
            continue

        # Extraire les paires présentes dans cette matrice
        matrix_combos = set(df.select(["start_vid", "end_vid"]).rows())

        # Vérifie les correspondances
        common_combos = matrix_combos & (combo_to_idx.keys() - found_set)

        for combo in common_combos:
            idx = combo_to_idx[combo]
            exists[idx] = True
            paths[idx] = str(txt_path)
            found_set.add(combo)

        # Optionnel : affiche une barre interne
        # for _ in tqdm(common_combos, leave=False, desc="✔️ Vérification"):
        #     pass

        if len(found_set) == len(combo_to_idx):
            print("Toutes les combinaisons ont été trouvées.")
            break
        

    # Ajouter les résultats dans le DataFrame final
    not_matching_concat = not_matching_concat.with_columns([
        pl.Series("exists_in_matrix", exists),
        pl.Series("matrix_path", paths)
    ])

    return not_matching_concat


In [27]:
if __name__ == "__main__":
    result_df = check_combinations(matrix_root=r"D:\Matrices-Brutes",
                        not_matching_concat=not_matching_concat)
    print(result_df)

🔍 Analyse de 2800 matrices pour 214 combinaisons...


Lecture des matrices .txt:   0%|          | 0/2800 [00:00<?, ?fichier/s]C:\Users\e_koffie\AppData\Local\Temp\ipykernel_20912\1763528989.py:40: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  df = pl.read_csv(
Lecture des matrices .txt: 100%|█████████▉| 2796/2800 [2:38:40<00:13,  3.41s/fichier]  

Toutes les combinaisons ont été trouvées.
shape: (214, 7)
┌──────────────┬──────────────┬──────────┬────────┬───────────────┬────────────────┬───────────────┐
│ Idloc_start  ┆ Idloc_end    ┆ id_start ┆ id_end ┆ combination   ┆ exists_in_matr ┆ matrix_path   │
│ ---          ┆ ---          ┆ ---      ┆ ---    ┆ ---           ┆ ix             ┆ ---           │
│ str          ┆ str          ┆ i64      ┆ i64    ┆ struct[2]     ┆ ---            ┆ str           │
│              ┆              ┆          ┆        ┆               ┆ bool           ┆               │
╞══════════════╪══════════════╪══════════╪════════╪═══════════════╪════════════════╪═══════════════╡
│ 010020298003 ┆ 010020298004 ┆ 10421    ┆ 15005  ┆ {10421,15005} ┆ true           ┆ E:\matrice-li │
│              ┆              ┆          ┆        ┆               ┆                ┆ gne-diarra\Ma │
│              ┆              ┆          ┆        ┆               ┆                ┆ tric…         │
│ 020270398012 ┆ 020270398013 ┆ 1

In [28]:
# Save result_df to a Parquet file
save_path = r"C:\Users\e_koffie\Documents\IAI_Project\SIMULATIONS\Data\result_df_ligne.parquet"
result_df.write_parquet(save_path)
print(f"result_df saved to {save_path}")

result_df saved to C:\Users\e_koffie\Documents\IAI_Project\SIMULATIONS\Data\result_df_ligne.parquet


In [29]:
true_count = result_df.filter(pl.col("exists_in_matrix") == True).height
false_count = result_df.filter(pl.col("exists_in_matrix") == False).height

print(f"True count: {true_count}")
print(f"False count: {false_count}")

True count: 214
False count: 0


In [30]:
not_matching_joined = not_matching.join(
    result_df.select(["Idloc_start", "Idloc_end", "exists_in_matrix", "matrix_path"]),
    on=["Idloc_start", "Idloc_end"],
    how="left"
)

In [31]:
not_matching_joined

Idloc_start,Idloc_end,id_start,id_end,exists_in_matrix,matrix_path
str,str,i64,i64,bool,str
"""010020298003""","""010020298004""",10421,15005,true,"""E:\matrice-ligne-diarra\Matric…"
"""020270398012""","""020270398013""",10470,14732,true,"""E:\matrice-ligne-diarra\Matric…"
"""020480298014""","""020480298016""",10163,10162,true,"""E:\matrice-ligne-diarra\Matric…"
"""020480298017""","""020480298996""",33104,33260,true,"""E:\matrice-ligne-diarra\Matric…"
"""020950298001""","""020950298002""",34016,29816,true,"""E:\matrice-ligne-diarra\Matric…"
…,…,…,…,…,…
"""321010201003""","""321010201997""",11632,10915,true,"""D:\Matrices-Brutes\matrice-lig…"
"""321010298012""","""321010298014""",3899,25293,true,"""D:\Matrices-Brutes\matrice-lig…"
"""330710198005""","""330710198006""",13403,14863,true,"""D:\Matrices-Brutes\matrice-lig…"


In [32]:
true_count_ = not_matching_joined.filter(pl.col("exists_in_matrix") == True).height
false_count_ = not_matching_joined.filter(pl.col("exists_in_matrix") == False).height

print(f"True count: {true_count_}")
print(f"False count: {false_count_}")

True count: 107
False count: 0


In [33]:
permutation_joined = not_matching_permuted.join(
    result_df.select(["Idloc_start", "Idloc_end", "exists_in_matrix", "matrix_path"]),
    on=["Idloc_start", "Idloc_end"],
    how="left"
)

In [34]:
permutation_joined

Idloc_start,Idloc_end,id_start,id_end,exists_in_matrix,matrix_path
str,str,i64,i64,bool,str
"""010020298004""","""010020298003""",15005,10421,true,"""E:\matrice-ligne-diarra\Matric…"
"""020270398013""","""020270398012""",14732,10470,true,"""E:\matrice-ligne-diarra\Matric…"
"""020480298016""","""020480298014""",10162,10163,true,"""E:\matrice-ligne-diarra\Matric…"
"""020480298996""","""020480298017""",33260,33104,true,"""E:\matrice-ligne-diarra\Matric…"
"""020950298002""","""020950298001""",29816,34016,true,"""E:\matrice-ligne-diarra\Matric…"
…,…,…,…,…,…
"""321010201997""","""321010201003""",10915,11632,true,"""D:\Matrices-Brutes\matrice-lig…"
"""321010298014""","""321010298012""",25293,3899,true,"""D:\Matrices-Brutes\matrice-lig…"
"""330710198006""","""330710198005""",14863,13403,true,"""D:\Matrices-Brutes\matrice-lig…"


In [35]:
true_count__ = permutation_joined.filter(pl.col("exists_in_matrix") == True).height
false_count__ = permutation_joined.filter(pl.col("exists_in_matrix") == False).height

print(f"True count: {true_count__}")
print(f"False count: {false_count__}")

True count: 107
False count: 0


In [36]:
def reconstituer_itineraires_par_ligne(
    result_df: pl.DataFrame,
    route_loc_path: str,
    liste_loc_path: str
) -> pl.DataFrame:
    """
    Reconstitue les itinéraires pour chaque ligne de result_df à partir de son matrix_path.
    """
    # Charger les tronçons et localités
    route_loc = pd.read_excel(route_loc_path)
    route_loc["gid"] = route_loc["gid"].astype(str)
    route_loc = pl.from_pandas(route_loc)

    liste_loc = pd.read_excel(liste_loc_path, dtype={"idloc_iai": str})
    liste_loc = pl.from_pandas(liste_loc)
    liste_loc = liste_loc.with_columns([pl.col("id").cast(pl.Int64)])

    itineraire_list = []

    print("Traitement ligne par ligne de result_df...")
    for row in tqdm(result_df.iter_rows(named=True), total=result_df.height):
        id_start = row["Idloc_start"]
        id_end = row["Idloc_end"]
        matrix_path = row["matrix_path"]

        try:
            matrix = pl.read_csv(matrix_path, separator=";", dtypes={"gid": pl.Utf8})
            matrix = matrix.join(route_loc, on="gid", how="left")
            matrix = matrix.select(["start_vid", "end_vid", "gid", "stat_voie"])
            matrix = matrix.filter((~pl.col("start_vid").is_null()) & (~pl.col("end_vid").is_null()))

            # Trouver les vids correspondant aux localités
            vid_start = liste_loc.filter(pl.col("idloc_iai") == id_start).select("id").item()
            vid_end = liste_loc.filter(pl.col("idloc_iai") == id_end).select("id").item()

            # Extraire les lignes concernées
            matching = matrix.filter((pl.col("start_vid") == vid_start) & (pl.col("end_vid") == vid_end))
            if matching.is_empty():
                itineraire_list.append("NA")
            else:
                gids = matching.select("gid").to_series().to_list()
                itineraire_list.append(";".join(gids))

        except Exception as e:
            print(f"Erreur ligne {id_start} → {id_end} dans {matrix_path} : {e}")
            itineraire_list.append("NA")

    # Ajouter à result_df
    result_df = result_df.with_columns([
        pl.Series("itineraire", itineraire_list)
    ])

    return result_df


In [37]:
permutation_itineraires = reconstituer_itineraires_par_ligne(
    result_df=permutation_joined,
    route_loc_path= r"C:\Users\e_koffie\Documents\IAI_Project\IAI_PROJECT\Data\Fichiers_Routes_Localites\route_27052024_VF.xlsx",
    liste_loc_path= r"C:\Users\e_koffie\Documents\IAI_Project\IAI_PROJECT\Data\Fichiers_Routes_Localites\localite_noeuds_02102024_VF_2.xls"
)

Traitement ligne par ligne de result_df...


  0%|          | 0/107 [00:00<?, ?it/s]C:\Users\e_koffie\AppData\Local\Temp\ipykernel_20912\3925172752.py:27: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  matrix = pl.read_csv(matrix_path, separator=";", dtypes={"gid": pl.Utf8})
100%|██████████| 107/107 [03:31<00:00,  1.97s/it]


In [38]:
permutation_itineraires

Idloc_start,Idloc_end,id_start,id_end,exists_in_matrix,matrix_path,itineraire
str,str,i64,i64,bool,str,str
"""010020298004""","""010020298003""",15005,10421,true,"""E:\matrice-ligne-diarra\Matric…","""40347"""
"""020270398013""","""020270398012""",14732,10470,true,"""E:\matrice-ligne-diarra\Matric…","""40185"""
"""020480298016""","""020480298014""",10162,10163,true,"""E:\matrice-ligne-diarra\Matric…","""36199"""
"""020480298996""","""020480298017""",33260,33104,true,"""E:\matrice-ligne-diarra\Matric…","""11558"""
"""020950298002""","""020950298001""",29816,34016,true,"""E:\matrice-ligne-diarra\Matric…","""14264"""
…,…,…,…,…,…,…
"""321010201997""","""321010201003""",10915,11632,true,"""D:\Matrices-Brutes\matrice-lig…","""40777"""
"""321010298014""","""321010298012""",25293,3899,true,"""D:\Matrices-Brutes\matrice-lig…","""16772"""
"""330710198006""","""330710198005""",14863,13403,true,"""D:\Matrices-Brutes\matrice-lig…","""40166"""


In [39]:
def add_temps_parcours_column(permutation_itineraires: pl.DataFrame, route_loc_path: str) -> pl.DataFrame:
    """
    Ajoute une colonne 'temps_parcours' à permutation_itineraires.
    Pour chaque itinéraire (séparé par ';'), récupère les temps_1 associés dans le fichier route_loc_path,
    puis fait la somme pour chaque ligne.
    """
    # Charger le fichier route (doit contenir les colonnes 'gid' et 'temps_1')
    route_loc = pd.read_excel(route_loc_path, dtype={"gid": str})
    # On s'assure que 'gid' est str pour la jointure
    route_loc["gid"] = route_loc["gid"].astype(str)
    # Créer un dictionnaire pour accès rapide
    gid_to_temps = dict(zip(route_loc["gid"], route_loc["temps_1"]))

    def compute_temps_parcours(itineraire: str):
        if not isinstance(itineraire, str) or itineraire == "NA" or itineraire.strip() == "":
            return None
        gids = [gid for gid in itineraire.split(";") if gid]
        try:
            temps = [float(gid_to_temps.get(gid, 0)) for gid in gids]
            return sum(temps)
        except Exception:
            return None

    # Appliquer la fonction à chaque ligne
    temps_parcours = permutation_itineraires["itineraire"].to_list()
    temps_parcours_sum = [compute_temps_parcours(itin) for itin in temps_parcours]

    # Ajouter la colonne au DataFrame Polars
    permutation_itineraires = permutation_itineraires.with_columns(
        pl.Series("temps_parcours", temps_parcours_sum)
    )
    return permutation_itineraires

In [40]:
permutation_iti = add_temps_parcours_column(permutation_itineraires, 
                                            route_loc_path= r"C:\Users\e_koffie\Documents\IAI_Project\IAI_PROJECT\Data\Fichiers_Routes_Localites\route_27052024_VF.xlsx")

In [41]:
permutation_iti

Idloc_start,Idloc_end,id_start,id_end,exists_in_matrix,matrix_path,itineraire,temps_parcours
str,str,i64,i64,bool,str,str,f64
"""010020298004""","""010020298003""",15005,10421,true,"""E:\matrice-ligne-diarra\Matric…","""40347""",18.488121
"""020270398013""","""020270398012""",14732,10470,true,"""E:\matrice-ligne-diarra\Matric…","""40185""",518.819243
"""020480298016""","""020480298014""",10162,10163,true,"""E:\matrice-ligne-diarra\Matric…","""36199""",267.980129
"""020480298996""","""020480298017""",33260,33104,true,"""E:\matrice-ligne-diarra\Matric…","""11558""",124.040621
"""020950298002""","""020950298001""",29816,34016,true,"""E:\matrice-ligne-diarra\Matric…","""14264""",249.990341
…,…,…,…,…,…,…,…
"""321010201997""","""321010201003""",10915,11632,true,"""D:\Matrices-Brutes\matrice-lig…","""40777""",1587.698493
"""321010298014""","""321010298012""",25293,3899,true,"""D:\Matrices-Brutes\matrice-lig…","""16772""",13015.174213
"""330710198006""","""330710198005""",14863,13403,true,"""D:\Matrices-Brutes\matrice-lig…","""40166""",390.906494


In [42]:
permuted = permutation_iti.select([
    pl.col("Idloc_end").alias("Idloc_start"),
    pl.col("Idloc_start").alias("Idloc_end"),
    pl.col("temps_parcours")
])

In [43]:
permuted

Idloc_start,Idloc_end,temps_parcours
str,str,f64
"""010020298003""","""010020298004""",18.488121
"""020270398012""","""020270398013""",518.819243
"""020480298014""","""020480298016""",267.980129
"""020480298017""","""020480298996""",124.040621
"""020950298001""","""020950298002""",249.990341
…,…,…
"""321010201003""","""321010201997""",1587.698493
"""321010298012""","""321010298014""",13015.174213
"""330710198005""","""330710198006""",390.906494


In [44]:
df_polars

Idloc_start,Idloc_end,temps_parcours
str,str,f64
"""150210398003""","""230140198041""",956047.892538
"""121100298003""","""310410401005""",779382.446285
"""080700298004""","""290330298014""",339317.691312
"""101000398011""","""280160198002""",957028.64086
"""110530298043""","""200150598996""",681183.361829
…,…,…
"""040130401032""","""210570498005""",279773.304878
"""150210301006""","""190460298014""",1.0171e6
"""160450198004""","""220320198019""",895553.817122


In [45]:
# Concatenate df_polars and permuted, then sort by Idloc_start and Idloc_end
combined_df = pl.concat([df_polars, permuted]).sort(["Idloc_start", "Idloc_end"])

In [47]:
# Drop duplicated rows in combined_df based on all columns
combined_df = combined_df.unique()

In [48]:
combined_df = combined_df.sort(["Idloc_start", "Idloc_end"])

In [49]:
combined_df

Idloc_start,Idloc_end,temps_parcours
str,str,f64
"""010020201001""","""010020201003""",36446.455987
"""010020201001""","""010020201004""",49300.388925
"""010020201001""","""010020201005""",39694.639219
"""010020201001""","""010020201007""",44502.637763
"""010020201001""","""010020201008""",37004.368235
…,…,…
"""330840498008""","""330840498010""",37581.33599
"""330840498008""","""330840498011""",48910.70363
"""330840498009""","""330840498010""",5815.175222


In [50]:
# Save combined_df as Parquet
combined_df.write_parquet(r"C:\Users\e_koffie\Documents\IAI_Project\SIMULATIONS\Data\dt_matrix_ligne_VF.parquet")

# Save permutation_iti as Parquet
permutation_iti.write_parquet(r"C:\Users\e_koffie\Documents\IAI_Project\SIMULATIONS\Data\dt_matrix_ligne_INTERM.parquet")